<a href="https://colab.research.google.com/github/hebatallah/constructionBERT/blob/master/classify_cause.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 1.3MB/s 
     |████████████████████████████████| 6.7MB 34.7MB/s 
     |████████████████████████████████| 10.0MB 52.7MB/s 
     |████████████████████████████████| 983kB 49.7MB/s 
     |████████████████████████████████| 245kB 59.7MB/s 
     |████████████████████████████████| 1.8MB 50.6MB/s 
     |████████████████████████████████| 501kB 56.9MB/s 
     |████████████████████████████████| 870kB 35.9MB/s 
     |████████████████████████████████| 1.0MB 44.2MB/s 
     |████████████████████████████████| 3.7MB 46.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.11.3-cp36-none-any.whl size=25213801 sha256=179cd7e2cb831babba51f77727c98322fc866f2303f6799a0323829b94e24c91
  Stored in directory: /root/.cache/pip/wheels/c7/6f/71/4c5097ea14a1e7791668a96c585d7fc81c45f0eb2e7a94b681
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=479ab8f8429e3e775a163080e0de94cada2a779d5a3f6b5c6df0da3e8d2fdb51
  Stored in direc

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [5]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/Tixierae/WECD/master/classification_data_set.csv

--2020-03-23 20:05:56--  https://raw.githubusercontent.com/Tixierae/WECD/master/classification_data_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5426143 (5.2M) [text/plain]
Saving to: ‘classification_data_set.csv’

classification_data 100%[===================>]   5.17M  --.-KB/s    in 0.05s   

2020-03-23 20:05:57 (102 MB/s) - ‘classification_data_set.csv’ saved [5426143/5426143]



In [10]:
from ktrain import text
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

# load classification data set
full_set = pd.read_csv('classification_data_set.csv', sep=',')

narrative = full_set['narrative'].tolist()
additional_info = full_set['additional_info'].tolist()

full_set['cause'] = list(map(lambda tdl: re.search('Cause: (.+?)Fat', str(tdl)).group(1) if re.search('Cause:(.+?)Fat', str(tdl)) != None else 'None', full_set['additional_info'].tolist()))
cause = full_set['cause'].tolist()
  
full_set = full_set[full_set['cause'] != 'None']

lb = LabelEncoder()
full_set['label_cause'] = lb.fit_transform(full_set['cause'])

full_set = full_set[full_set['label_cause'].isin(['11','48','30','10','66','4','39','68','24','25','14','8','17','0','67','38','13','50'])] 
 
vc = full_set['label_cause'].value_counts()
print(vc)

target_names = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

full_set['cause'] = full_set['cause'].replace('Exterior carpentry', 0)
full_set['cause'] = full_set['cause'].replace('Roofing', 1)
full_set['cause'] = full_set['cause'].replace('Interior carpentry', 2)
full_set['cause'] = full_set['cause'].replace('Excavation', 3)
full_set['cause'] = full_set['cause'].replace('Temporary work (buildings, facilities)', 4)
full_set['cause'] = full_set['cause'].replace('Demolition', 5)
full_set['cause'] = full_set['cause'].replace('Paving', 6)
full_set['cause'] = full_set['cause'].replace('Trenching, installing pipe', 7)
full_set['cause'] = full_set['cause'].replace('Installing equipment (HVAC and other)', 8)
full_set['cause'] = full_set['cause'].replace('Installing interior walls, ceilings, doors', 9)
full_set['cause'] = full_set['cause'].replace('Exterior painting' , 10)
full_set['cause'] = full_set['cause'].replace('Erecting structural steel', 11)
full_set['cause'] = full_set['cause'].replace('Forming', 12)
full_set['cause'] = full_set['cause'].replace('Backfilling and compacting', 13)
full_set['cause'] = full_set['cause'].replace('Traffic protection', 14)
full_set['cause'] = full_set['cause'].replace('Other Activities-Post Decking Detail Work', 15)
full_set['cause'] = full_set['cause'].replace('Exterior masonry', 16)
full_set['cause'] = full_set['cause'].replace('Site clearing and grubbing', 17)


X_train, X_test, Y_train, Y_test = train_test_split(full_set['narrative'].tolist(), full_set['cause'].tolist(), stratify = full_set['cause'], test_size=0.2, random_state=0) 



11    557
48    420
30    295
10    267
66    260
4     254
39    246
68    230
24    204
25    185
14    139
8     129
17    127
0     123
67    120
38    119
13    112
50    101
Name: label_cause, dtype: int64


In [0]:
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=512, class_names=target_names)
trn = t.preprocess_train(X_train, Y_train)
val = t.preprocess_test(X_test, Y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)
learner.fit_onecycle(2e-5, 5)


preprocessing train...
language: en
train sequence lengths:
	mean : 106
	95percentile : 259
	99percentile : 412


preprocessing test...
language: en
test sequence lengths:
	mean : 105
	95percentile : 245
	99percentile : 372





begin training using onecycle policy with max lr of 2e-05...
Train for 519 steps, validate for 130 steps
Epoch 1/5
519/519 [==============================] - 431s 831ms/step - loss: 2.6516 - accuracy: 0.1974 - val_loss: 2.2193 - val_accuracy: 0.3136
Epoch 2/5
519/519 [==============================] - 419s 808ms/step - loss: 1.9419 - accuracy: 0.4068 - val_loss: 1.7517 - val_accuracy: 0.4473
Epoch 3/5
519/519 [==============================] - 419s 808ms/step - loss: 1.5522 - accuracy: 0.5296 - val_loss: 1.6581 - val_accuracy: 0.4833
Epoch 4/5
519/519 [==============================] - 420s 808ms/step - loss: 1.1668 - accuracy: 0.6579 - val_loss: 1.6208 - val_accuracy: 0.5129
Epoch 5/5
519/519 [==============================] - 420s 809ms/step - loss: 0.8764 - accuracy: 0.7592 - val_loss: 1.5830 - val_accuracy: 0.5347


In [0]:
matrix = learner.validate(val_data=val)

              precision    recall  f1-score   support

           0       0.51      0.65      0.57       112
           1       0.78      0.70      0.74        84
           2       0.57      0.58      0.57        59
           3       0.45      0.42      0.43        53
           4       0.26      0.29      0.27        52
           5       0.54      0.51      0.53        51
           6       0.56      0.69      0.62        49
           7       0.48      0.54      0.51        46
           8       0.45      0.51      0.48        41
           9       0.64      0.49      0.55        37
          10       0.92      0.86      0.89        28
          11       0.54      0.50      0.52        26
          12       0.41      0.44      0.42        25
          13       0.47      0.36      0.41        25
          14       0.58      0.58      0.58        24
          15       0.00      0.00      0.00        24
          16       0.52      0.59      0.55        22
          17       0.42    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
# Save TF model
learner.save_model('./bert-base-construction-cause-cased')